# Introducing Solara

### Goal of this notebook 

After doing all this work, we should also review another approach to the entire problem of developing web apps with Python. This approach is called Solara, and it is both a Python library that allows you to create web applications using Python code only and a server to deploy those web apps.


### Steps you will take in this notebook

1. Learn about *reactive* variables and solara *components*.
2. Build a version of our dashboard using solara.
3. Briefly discuss how a solara server differs from a voila server.
4. Discuss the steps to deploy a solara app on the [ploomber](https://www.platform.ploomber.io/) service.


In [ ]:
#| default_exp app 

In [ ]:
#| export
from pathlib import Path

import pandas as pd
import solara
from ipydatagrid import DataGrid
from matplotlib.figure import Figure
from matplotlib import pyplot as plt
from scipy.signal import savgol_filter


## Read the data 

In [ ]:
#| export

DATA_DIR = 'data'
DATA_FILE = 'land-ocean-temp-index.csv'

original_df = pd.read_csv(Path(DATA_DIR) / DATA_FILE, escapechar='#')
year_range_input = (min(original_df["Year"]), max(original_df["Year"]))

## Set up the *reactive* variables and controls

These allow solara to handle updating controls as values change.

### Reactive variables for the controls

The cell below defines three *reactive* variables. These are variables whose changes will be monitored for
changes by solara. Variables defined with `solara.reactive` are typically global variables used to manage the state of the application. It is possible to define reactive variables that are local to a function by using `solara.use_state` or `solar.use_reactive`.

The argument to `solara.reactive` is the initial value of the variable. That initial value can have any type.

In [ ]:
#| export
year_range = solara.reactive(year_range_input)
window_size = solara.reactive(2)
polynomial_order = solara.reactive(1)


In [ ]:
# Print one of the values -- just like for an ipywidgets you use .value
print(year_range.value)

### Solara components for controls

The cell below defines, in one function, a column of controls: 3 sliders, one for `year_range`, one for `window_size` and one for `polynomial_order`.

*Components* are the building blocks out of which a solara application is built. You can make a function you write a component in solara by using the decorator `@solara.component`. 

In [ ]:
#| export
# Here we define our own component, called controls. A component can take arguments, 
# though this one does not.
@solara.component
def controls():
    """
    This panel contains the year_range, window_size and polynomial_order controls.
    """
    # solar.Column() is another component defined in solara itself. Everything in the 
    # with block is arranged in a column.
    with solara.Column() as crtl:
        # SliderRangeInt is another solara component
        solara.SliderRangeInt(
            "Range of years", 
            # The line below is key -- it connects the slider to the reactive variable year_range
            value=year_range, 
            min=year_range_input[0],
            max=year_range_input[1],
        )
    
        solara.SliderInt(
            "Window size",
            # Link this slider to window_size
            value=window_size,
            min=2,
            max=100
        )
        solara.SliderInt(
            "Polynomial order",
            # Link this slider to polynomial_order
            value=polynomial_order,
            min=1,
            max=10
        )
    # If there is a single displayable component in the function then solara will display that,
    # otherwise it renders the return value.
    return crtl



### Another component for limiting polynomial order

The component below does not display anything on the screen. Instead, it checks for consistency between the `window_size` and `polynomial_order`. Because we have defined both of those as *reactive* variables, solara will automatically call this component, as long as we include a call to it in one of the components is displayed. We'll put that call in our "main" dashboard below, but it could be worked into the definition of our controls instead.

In [ ]:
#| export

# Registering as a component ensures this is called when either reactive variable's 
# value changes.
@solara.component
def check_poly_order():
    if polynomial_order.value > 10 or polynomial_order.value >= window_size.value:
        polynomial_order.value = min(window_size.value - 1, 10)

### Reactive variables and components for the data

The argument of `solara.reactive` can be anything, including a Pandas data frame. Declaring this as reactive ensures that solara responds when the selected data changes.

In [ ]:
#| export
selected_df = solara.reactive(original_df.copy())

The component below also does not display any data. The only thing it does is calculate a smoothing column and update `selected_df`. This will automatically be called when `year_range`, `window_size` or `polynomial_order` changes.

In [ ]:
#| export

@solara.component 
def selected_data():
    """
    This component only updates the selected data. Since selected_df is a reactive 
    variable, any component which 1) uses selected_df and 2) is rendered in a UI component
    will automatically be updated.
    """
    original_df['Smoothed Data'] = savgol_filter(original_df['Temperature'],
                                                 window_size.value,
                                                 polynomial_order.value).round(decimals=3)
    selected_df.value = original_df[(original_df['Year'] >= year_range.value[0])
                               & (original_df['Year'] <= year_range.value[1])]


## Make the plot

Either the pyplot or object interface to matplotlib can be used. If pyplot is used, then the plot should be closed after drawing it so that you do not end up with a bunch of open (but inaccessible) plots.

Since we declared `selected_df` as a reactive variable, the plot is redrawn whenever its value changes.

In [ ]:
#| export 

@solara.component
def draw_plot():
    plt.xlabel('Year')
    plt.ylabel('Temperature Anomalies over Land w.r.t. 1951-80 (˚C)')
    plt.title('Global Annual Mean Surface Air Temperature Change')

    plt.plot(selected_df.value['Year'], selected_df.value['Temperature'], label='Raw Data')
    plt.plot(selected_df.value['Year'], selected_df.value['Smoothed Data'], label='Smoothed Data')
    plt.legend()
    plt.show()
    plt.close()


## Defining the dashboard

In the cell below we define the dashboard. We could call it anything we want, but when running it as a dashboard using `solara-server` if there is an object called `Page`, then that is what will be rendered in the browser.

The overall layout will have one row with two columns. The first column has the controls and the second column has the data and graph.

In [ ]:
#| export
@solara.component
def Page():
    # These first two components are called here so that solara knows it should call them 
    # when changes occur in any of the reactive variables used in those components.
    check_poly_order()
    selected_data()
    
    # We make a row, which will end up with two columns 
    with solara.Row():
        # Here we define the left column and restrict its width to 500px.
        with solara.Column(style=dict(width="500px")):
            # Get some extra space at the top...
            solara.Text("\n\n")
            # Here we use the controls component we defined above.
            controls()
        # Make column 2 with the data and graph. This column will use whatever space
        # is available that the first column doesn't use.
        with solara.Column():
            # Display the data. The Details component is a collapsible component sort of like
            # an accordion. Its child is an ipydatagrid.DataGrid, like we used previously.
            # There is another option built in to solara called solara.DataFrame with similar 
            # functionality.
            solara.Details(
                summary="Click to show data",
                children=[DataGrid(selected_df.value)]
            )
            # This draws the plot. Solara undestands that this needs to be redrawn whenever selected_df
            # changes.
            draw_plot()


## Display the dashboard

In [ ]:
Page()

## Export with nbdev

In [ ]:
from nbdev.export import nb_export

nb_export('04c_solara.ipynb', 'dashboard_solara')

## Displaying the dashboard with solara

Solara includes both the Python framework for writing widgets that we have talked about and a server for displaying notebooks. Copy/paste this into a terminal to run this dashboaard using solara:

```bash
solara run 04c_solara.ipynb
```

# Be careful, solara can be an effective foot gun

See if you can spot the error in the code below -- why doesn't the range slider behave as expected?


In [ ]:
year_range_ex2 = solara.reactive((1880, 2023))

@solara.component
def BadSlider():
    solara.SliderRangeInt(
        "Some integer range", 
        value=year_range_ex2, 
        min=year_range_ex2.value[0], 
        max=year_range_ex2.value[1],
        tick_labels=True,
        step=5
    )

BadSlider()

## Solara exercises

### 1. Add the text widgets

Add the text widgets from the dashboard to this solara example. Use the [solara `HTML` component](https://solara.dev/documentation/components/output/html) to display the text.


### 2. Improve the controls consistency check

The window size should really be at least one less than the range of years displayed. In other words, it does not make sense to use a window size of 100 if you are only displaying 50 years of data. It does not crash here when you do that because the smoothed column is called from the full data set.